In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime

por ter usado `&dedupValue=25&dedupField=url` e diferentes aliases, há informação repetida

**filtrar repetidos e textos que não mencionem nenhum alias**

problemas ultrapassados:

- API has the following usage limits (250req/min, error 429): `time.sleep(60)`

- API error 404 for some urls: return 0 (False) and skip it

- extrair o texto demora muito: filtrar e salvar coluna a coluna

nota: podia ter feito já online o processamento do texto, mas não queria estar dependente do wifi

# data02.parquet

In [28]:
def extracText(linkToExtractedText):
    # Infinite loop to handle retry logic in case of 429 Too Many Requests
    while True:
        response = requests.get(linkToExtractedText)
        status_code = response.status_code
        
        if status_code == 200:
            # If the request is successful (200 OK), return the extracted text
            soup = BeautifulSoup(response.content, "html.parser")
            return soup.get_text()
        elif status_code == 429:
            # Handle 429 Too Many Requests by reading the Retry-After header
            print(" (...)", end = "")
            time.sleep(60)  # Pause execution for the retry period
        elif status_code == 404:
            return 0
        else:
            # For any other status codes (e.g., 500, ...), print the status and break the loop
            print(f"Request failed with status code {status_code}. Link was {linkToExtractedText}")
            break

# Function to process each column
def filterColumn(column, aliases):
    """aliases in text, repeated text and extract text"""
    global stats
    filtered_column = []

    for row in aliases.index:
        filtered_cell = []
        seen_text = set()
        print(f"; {row}", end = "")
        for i in column.loc[row]:
            
            # Extract text from 'linkToExtractedText'
            text = extracText(i['linkToExtractedText'])
                

            # Skip if the text has already been processed
            if text in seen_text:
                stats["duplicate"] += 1
                continue

            elif not text: #ERROR 404
                stats["404"] += 1
                continue
            
            # Check if any alias is found in the text
            elif any(alias.lower() in text.lower() for alias in aliases.loc[row]):
                i["ExtractedText"] = text  # Add extracted text to the record
                
                # Remove unwanted fields
                i.pop('linkToExtractedText', None)
                
                # Append the processed record
                filtered_cell.append(i)
                
                # Mark this text as processed
                seen_text.add(text)

        filtered_column.append(filtered_cell)
                
    return filtered_column


def processColumns(col_to_proc):
    print(f"Starting: {datetime.now()}")
    try:
        # continuar df criada
        df = pd.read_parquet("data02.parquet")
    except:
       # criar df para trabalhar
       df = pd.read_parquet("data01.parquet").to_parquet("data02.parquet")
       df = pd.read_parquet("data02.parquet")
    for column in col_to_proc:
        has_link = "linkToExtractedText" in df.iloc[-1][column][-1]
        has_extracText = "ExtractedText" in df.iloc[-1][column][-1]
        if not has_link and has_extracText:
            print(f"\n{column} already done. Skipping.")
        else:
            print(f"\nProcessing {column}", end = ": ")
            df[column] = filterColumn(df[column], df["aliases"])
            df.to_parquet("data02.parquet")
    print(f"\nEnded: {datetime.now()}.")

stats = {"404": 0, "duplicate": 0}
processColumns(["api.2000", "api.2003", "api.2006", "api.2009", "api.2012", "api.2015", "api.2018"])
print(stats)

Starting: 2024-10-09 13:21:14.857682

api.2000 already done. Skipping.

api.2003 already done. Skipping.

api.2006 already done. Skipping.

api.2009 already done. Skipping.

api.2012 already done. Skipping.

api.2015 already done. Skipping.

api.2018 already done. Skipping.

Ended: 2024-10-09 13:21:15.067062.
{'404': 0, 'duplicate': 0}


In [32]:
pd.read_parquet("data02.parquet").map(lambda x: len(x)) - pd.read_parquet("data01.parquet").map(lambda x: len(x))

,aliases,api.2000,api.2003,api.2006,api.2009,api.2012,api.2015,api.2018
companies,,,,,,,,
Banco Comercial Português,0,-63,-50,-14,-64,-91,-211,-130
Galp Energia,0,-62,-156,-91,-113,-192,-287,-156
EDP,0,-53,-94,-33,-115,-156,-442,-224
Sonae,0,-62,-117,-40,-43,-106,-305,-170
Mota-Engil,0,-1,-16,-34,-31,-154,-232,-115


In [33]:
pd.read_parquet("data02.parquet").map(lambda x: len(x))

,aliases,api.2000,api.2003,api.2006,api.2009,api.2012,api.2015,api.2018
companies,,,,,,,,
Banco Comercial Português,2,90,191,169,497,983,1219,824
Galp Energia,2,66,233,181,469,964,1104,812
EDP,3,80,245,140,538,1076,1528,872
Sonae,2,130,318,239,459,1109,1400,1026
Mota-Engil,2,3,67,26,164,384,596,445
